In [1]:
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [7]:
cd drive/My Drive/OstProject


/content/drive/My Drive/OstProject


In [8]:
!ls


001_1.bmp  bestDiff.py		    IITDatabase  Results
01_L.bmp   comparing_two_images.py  Normalized


In [0]:
cd content/drive/My Drive/OstProject

[Errno 2] No such file or directory: 'content/drive/My Drive/OstProject'
/content


In [0]:
cd OstProject

/content/drive/My Drive/OstProject


In [0]:
!ls


001_1.bmp  bestDiff.py		    IITDatabase  Results
01_L.bmp   comparing_two_images.py  Normalized


In [0]:
!git clone https://github.com/terror26/OstProject.git

Cloning into 'OstProject'...
remote: Enumerating objects: 5256, done.
remote: Total 5256 (delta 0), reused 0 (delta 0), pack-reused 5256
Receiving objects: 100% (5256/5256), 226.72 MiB | 11.13 MiB/s, done.
Resolving deltas: 100% (305/305), done.
Checking out files: 100% (4497/4497), done.


In [9]:
import warnings
from PIL import Image
from skimage.measure import compare_ssim
from skimage.transform import resize
from scipy.stats import wasserstein_distance
from scipy.misc import imsave
from scipy.ndimage import imread
import numpy as np
import cv2
import os.path
##
# Globals
##

warnings.filterwarnings('ignore')

# specify resized image sizes
height = 2**10
width = 2**10

##
# Functions
##

def get_img(path, norm_size=True, norm_exposure=False):
  '''
  Prepare an image for image processing tasks
  '''
  # flatten returns a 2d grayscale array
  img = imread(path, flatten=True).astype(int)
  # resizing returns float vals 0:255; convert to ints for downstream tasks
  if norm_size:
    img = resize(img, (height, width), preserve_range=True)
  if norm_exposure:
    img = normalize_exposure(img)
  return img


def get_histogram(img):
  '''
  Get the histogram of an image. For an 8-bit, grayscale image, the
  histogram will be a 256 unit vector in which the nth value indicates
  the percent of the pixels in the image with the given darkness level.
  The histogram's values sum to 1.
  '''
  h, w = img.shape
  hist = [0.0] * 256
  for i in range(h):
    for j in range(w):
      hist[img[i, j]] += 1
  return np.array(hist) / (h * w) 


def normalize_exposure(img):
  '''
  Normalize the exposure of an image.
  '''
  img = img.astype(int)
  hist = get_histogram(img)
  # get the sum of vals accumulated by each position in hist
  cdf = np.array([sum(hist[:i+1]) for i in range(len(hist))])
  # determine the normalization values for each unit of the cdf
  sk = np.uint8(255 * cdf)
  # normalize each position in the output image
  height, width = img.shape
  normalized = np.zeros_like(img)
  for i in range(0, height):
    for j in range(0, width):
      normalized[i, j] = sk[img[i, j]]
  return normalized.astype(int)


def earth_movers_distance(path_a, path_b):
  '''
  Measure the Earth Mover's distance between two images
  @args:
    {str} path_a: the path to an image file
    {str} path_b: the path to an image file
  @returns:
    TODO
  '''
  img_a = get_img(path_a, norm_exposure=True)
  img_b = get_img(path_b, norm_exposure=True)
  hist_a = get_histogram(img_a)
  hist_b = get_histogram(img_b)
  return wasserstein_distance(hist_a, hist_b)


def structural_sim(path_a, path_b):
      '''
      Measure the structural similarity between two images
      @args:
    {str} path_a: the path to an image file
    {str} path_b: the path to an image file
      @returns:
    {float} a float {-1:1} that measures structural similarity
      between the input images
      '''

      img_a = get_img(path_a)
      img_b = get_img(path_b)
      sim, diff = compare_ssim(img_a, img_b, full=True)
      return sim


def pixel_sim(path_a, path_b):
  '''
  Measure the pixel-level similarity between two images
  @args:
    {str} path_a: the path to an image file
    {str} path_b: the path to an image file
  @returns:
    {float} a float {-1:1} that measures structural similarity
      between the input images
  '''
  img_a = get_img(path_a, norm_exposure=True)
  img_b = get_img(path_b, norm_exposure=True)
  return np.sum(np.absolute(img_a - img_b)) / (height*width) / 255


def sift_sim(path_a, path_b):
  '''
  Use SIFT features to measure image similarity
  @args:
    {str} path_a: the path to an image file
    {str} path_b: the path to an image file
  @returns:
    TODO
  '''
  # initialize the sift feature detector
  orb = cv2.ORB_create()

  # get the images
  img_a = cv2.imread(path_a)
  img_b = cv2.imread(path_b)

  # find the keypoints and descriptors with SIFT
  kp_a, desc_a = orb.detectAndCompute(img_a, None)
  kp_b, desc_b = orb.detectAndCompute(img_b, None)

  # initialize the bruteforce matcher
  bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

  # match.distance is a float between {0:100} - lower means more similar
  matches = bf.match(desc_a, desc_b)
  similar_regions = [i for i in matches if i.distance < 70]
  if len(matches) == 0:
    return 0
  return len(similar_regions) / len(matches)


1109

In [41]:
img_a = 'Normalized/020_1.bmp'
img_b = ''
folder = 'Normalized'

shifts = []
structsims = []
files = sorted(os.listdir(folder))
len(files)

for i in files:
    if i =='.DS_Store':
        continue
    img_a = folder + '/'+i
    print(img_a)
    x = Image.open(img_a)
    l,h = x.size
    if (l < 100 or h < 100) :      ## To be handled
        list.append(realshifts,list())
        list.append(realstructsims,list())
        continue
    shifts = []
    structsims = []
    for filename in files:
        if filename =='.DS_Store':          
            shifts.append(0)
            continue
        #print('i is '+filename)
        img_b = folder + '/'+filename
        x = Image.open(img_b)
        l,h = x.size
        if (l < 100 or h < 100) :      ## To be handled
            #structsims.append(structsims,0)
            shifts.append(0)
            continue
        #print('img_a is ' + img_a)

        #----------------- Just tryign the histogram one-------------
        #emd = earth_movers_distance(img_a, img_b)

        #structural_sim_1 = structural_sim(img_a,img_b)
        #list.append(structsims,emd)
    
        #pixel_sim = pixel_sim(img_a,img_b)
        #sift_sim = sift_sim(img_a,img_b)
        sift_sim_1 = sift_sim(img_a,img_b)
        
        #emd = earth_movers_distance(img_a,img_b)
        #print(structural_sim, pixel_sim, sift_sim, emd)
        #print('structural_sim is '+str(structural_sim_1))
        #print('sift is __ :' + str(sift_sim_1))
        shifts.append(sift_sim_1)
    realshifts.append(shifts)
    #list.append(realstructsims,structsims)



Normalized/001_1.bmp
Normalized/001_2.bmp
Normalized/001_3.bmp
Normalized/001_4.bmp
Normalized/001_5.bmp
Normalized/002_1.bmp
Normalized/002_2.bmp
Normalized/002_3.bmp
Normalized/002_4.bmp
Normalized/002_5.bmp
Normalized/003_1.bmp
Normalized/003_2.bmp
Normalized/003_3.bmp
Normalized/003_4.bmp
Normalized/003_5.bmp
Normalized/004_1.bmp
Normalized/004_2.bmp
Normalized/004_3.bmp
Normalized/004_4.bmp
Normalized/004_5.bmp
Normalized/005_1.bmp
Normalized/005_2.bmp
Normalized/005_3.bmp
Normalized/005_4.bmp
Normalized/005_5.bmp
Normalized/006_1.bmp
Normalized/006_2.bmp
Normalized/006_3.bmp
Normalized/006_4.bmp
Normalized/006_5.bmp
Normalized/007_1.bmp
Normalized/007_2.bmp
Normalized/007_3.bmp
Normalized/007_4.bmp
Normalized/007_5.bmp
Normalized/008_1.bmp
Normalized/008_2.bmp
Normalized/008_3.bmp
Normalized/008_4.bmp
Normalized/008_5.bmp
Normalized/009_1.bmp
Normalized/009_2.bmp
Normalized/009_3.bmp
Normalized/009_4.bmp
Normalized/009_5.bmp
Normalized/010_1.bmp
Normalized/010_2.bmp
Normalized/01

In [0]:
import csv
with open("results.csv","w+") as my_csv:
    csvWriter = csv.writer(my_csv,delimiter=',')
    csvWriter.writerows(realshifts)

In [75]:
print(realshifts[1])

[0, 1.0, 0.9957983193277311, 0.9953051643192489, 0.9848484848484849, 0.8681318681318682, 0.8045112781954887, 0.875, 0.8854961832061069, 0.8796992481203008, 0.5960264900662252, 0.8294573643410853, 0.8449612403100775, 0.9203539823008849, 0.881578947368421, 0.9375, 0.6751592356687898, 0.75, 0.8089171974522293, 0.46153846153846156, 0.5754189944134078, 0.6139240506329114, 0.5882352941176471, 0.5886075949367089, 0.5180722891566265, 0.5592105263157895, 0.9096045197740112, 0.8372093023255814, 0.8659217877094972, 0.8166666666666667, 0.7659574468085106, 0.6973684210526315, 0.6666666666666666, 0.6666666666666666, 0.6035502958579881, 0.6114285714285714, 0.7272727272727273, 0.7410714285714286, 0.6370967741935484, 0.7413793103448276, 0.6935483870967742, 0.8807947019867549, 0.8509316770186336, 0.7528089887640449, 0.7087912087912088, 0.7719298245614035, 0.6024096385542169, 0.7605633802816901, 0.6436170212765957, 0.6906077348066298, 0.8592592592592593, 0.4787234042553192, 0.44148936170212766, 0.4293478

In [0]:
#checking ...
#cnt = 0
#for j in range(0,len(shifts)):
#    if shifts[j] == 1:
#            print(str(j))
#            cnt= cnt+1
#print('cnt is '+ str(cnt))
predictions = []
for k in range(0,1109):
  predictions.append(-1)
for i in range(0,len(realshifts)):
    cnt = 0
    temp = []
    for k in range(0,222):
      temp.append(0)
    for j in range(0,len(realshifts[i])):
        if realshifts[i][j] >= 0.7:
            temp[int((i-1)/5)] = temp[int((i-1)/5)] + 1;
            cnt= cnt+1
    for k in range(0,222):
      if(temp[k] > 1):
        predictions[i] = k


In [0]:
import pandas

pd = pandas.DataFrame(predictions)
pd.to_csv("result2.csv")


In [122]:
notpredict = 0
correct = 0
wrong = 0
for i in predictions:
  if(predictions[i] == -1 ):
    notpredict = notpredict + 1
  elif predictions[i] == int((i-1)/5):
    correct = correct +1;
  else:
    wrong = wrong +1;
print('wrongs'+str(wrong))
print('correct'+str(correct))
print('notpredict'+str(notpredict))

wrongs0
correct1035
notpredict74


In [124]:
print(predictions)

[-1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 14, 14, 14, 14, 14, 15, 15, 15, 15, 15, 16, 16, 16, -1, 16, 17, 17, 17, 17, 17, 18, 18, 18, 18, 18, 19, 19, 19, 19, 19, 20, 20, 20, 20, 20, 21, 21, 21, 21, 21, 22, 22, 22, 22, 22, 23, 23, 23, 23, -1, 24, 24, 24, 24, 24, 25, 25, 25, 25, 25, 26, 26, 26, 26, 26, 27, 27, 27, 27, 27, 28, 28, 28, 28, 28, 29, 29, 29, 29, 29, -1, 30, 30, -1, 30, -1, 31, 31, 31, 31, 32, 32, 32, 32, 32, 33, 33, 33, 33, 33, 34, 34, 34, 34, 34, 35, 35, 35, 35, 35, 36, 36, 36, 36, 36, 37, 37, 37, 37, 37, -1, 38, 38, 38, 38, 39, 39, 39, 39, 39, 40, 40, 40, 40, 40, 41, 41, 41, 41, 41, 42, 42, 42, 42, 42, 43, 43, 43, 43, 43, 44, 44, 44, 44, 44, 45, 45, 45, 45, 45, 46, 46, 46, 46, 46, 47, 47, 47, 47, 47, 48, 48, 48, 48, 48, 49, 49, 49, 49, 49, 50, 50, 50, 50, 50, 51, 51, 51, 51, 51, 52, 5